In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import string  
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to C:\Users\Gayathri
[nltk_data]     Palepu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Gayathri
[nltk_data]     Palepu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Gayathri
[nltk_data]     Palepu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
! pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
from transformers import pipeline
from datasets import load_dataset,load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
from datasets import load_dataset
dataset=load_dataset("cnn_dailymail",version="3.0.0")
print(f"Features in cnn_dailymail : {dataset['train'].column_names}")

Found cached dataset cnn_dailymail (C:/Users/Gayathri Palepu/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Features in cnn_dailymail : ['article', 'highlights', 'id']


In [6]:
sample_text=dataset['train'][1]['article'][:1000]
summaries={}
sample_text

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [7]:
def summary(text):
  return "\n".join(sent_tokenize(text)[:3])

In [8]:
from transformers import pipeline
import torch

In [9]:
sample_text=dataset['test'][1]['article'][:1000]

In [10]:
pipe=pipeline("summarization",model="facebook/bart-large-cnn")
pipe_out=pipe(sample_text)

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
summaries['bart']="\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries['bart']

'The dog was hit by a car, apparently whacked on the head with a hammer and then buried in a field.\nFour days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated.\nShe was found by a worker who took her to a vet for help.'

In [ ]:
from datasets import load_metric

In [ ]:
rouge_metric=load_metric('rouge')

C:\Users\Gayathri Palepu\AppData\Local\Temp\ipykernel_13112\2207925105.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric=load_metric('rouge')


In [ ]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
reference=dataset['test'][0]['highlights']
records=[]
for i in summaries:
  rouge_metric.add(prediction=summaries[i],reference=reference)
  score=rouge_metric.compute()
  rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
  print('rouge_dict',rouge_dict)
  records.append(rouge_dict)
pd.DataFrame.from_records(records,index=summaries.keys() )

rouge_dict {'rouge1': 0.13636363636363638, 'rouge2': 0.0, 'rougeL': 0.0909090909090909, 'rougeLsum': 0.11363636363636363}


,rouge1,rouge2,rougeL,rougeLsum
bart,0.136364,0.0,0.090909,0.113636


In [ ]:
! pip install -U sentence-transformers

     ---------------------------------------- 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
     ---------------------------------------- 1.1/1.1 MB 2.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b3def936b44a1190e0cf055d9027492985262cc6e1c9f58de7e5f87659e0ac7d
  Stored in directory: c:\users\gayathri palepu\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
! pip install bert-extractive-summarizer

     ---------------------------------------- 12.2/12.2 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 481.4/481.4 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 48.9/48.9 kB 309.0 kB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 181.6/181.6 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [ ]:
pip install --upgrade pip

In [ ]:
pip install tensorflow

In [ ]:
import pickle
myvar="savemodel.pkl"
pickle.dump(pipe,open(myvar,'wb'))

c:\Users\Gayathri Palepu\anaconda3\lib\site-packages\transformers\generation\tf_utils.py:446: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...bias_layer
......vars
.........0
...layers\tf_bart_main_layer
......vars
...layers\tf_bart_main_layer\decoder
......vars
...layers\tf_bart_main_layer\decoder\dropout
......vars
...layers\tf_bart_main_layer\decoder\embed_positions
......vars
.........0
...layers\tf_bart_main_layer\decoder\embed_tokens
......vars
.........0
...layers\tf_bart_main_layer\decoder\layernorm_embedding
......vars
.........0
.........1
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer
......vars
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer\activation_dropout
......vars
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer\dropout
......vars
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer\encoder_attn
......vars
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer\encoder_attn\dropout
......vars
...layers\tf_bart_main_layer\decoder\layers\tf_bart_decoder_layer\encod

In [ ]:
pip install numpy --upgrade

     -------------------------------------- 14.9/14.9 MB 665.9 kB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Gayathri Palepu\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
pip install --upgrade Pillow

     ---------------------------------------- 2.5/2.5 MB 291.4 kB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall numpy

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.
